In [52]:
%pip install backtesting

Note: you may need to restart the kernel to use updated packages.


In [53]:
import numpy as np
import pandas as pd
from backtesting import Backtest
# 
# List of file names
files = ['w1Jan.csv', 'w2Jan.csv', 'w3Jan.csv','w4Jan.csv','goldRush.csv']

# Create an empty list to store the dataframes
df_list = []

# Loop through the list of files and read each file into a dataframe
for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

# Concatenate the dataframes in the list
data = pd.concat(df_list)

# Reset the index
data.reset_index(drop=True, inplace=True)
# Load the stock market data
# data = pd.read_csv('goldRush.csv')

def calculate_atr(df, period):
    true_range = np.maximum(df['High'] - df['Low'], df['High'] - df['Close'].shift(1), df['Close'].shift(1) - df['Low'])
    atr = true_range.rolling(window=period).mean()
    return atr

def atr_trailing_stop(src, atr):
    x_atr_trailing_stop = 0.0
    # n_loss = atr * 1
    result = []
    src=src.values
    atr=atr.values
    for i in range(len(src)):
        n_loss = atr[i]
        if i == 0:
            result.append(x_atr_trailing_stop)
        else:
            if src[i] > src[i-1] and src[i-1] > x_atr_trailing_stop:
                x_atr_trailing_stop = max(x_atr_trailing_stop, src[i] - n_loss)
            elif src[i] < src[i-1] and src[i-1] < x_atr_trailing_stop:
                x_atr_trailing_stop = min(x_atr_trailing_stop, src[i] + n_loss)
            elif src[i] > src[i-1]:
                x_atr_trailing_stop = src[i] - n_loss
            else:
                x_atr_trailing_stop = src[i] + n_loss
            result.append(x_atr_trailing_stop)
    return result

def ema(close_price):
    ema = close_price[0]
    ema_values = [ema]
    for i in range(1, len(close_price)):
        ema = ema * (1 - 1/1) + close_price[i] * (1/1)
        ema_values.append(ema)
    return ema_values


# Calculate the Average True Range (ATR) indicator
atr = calculate_atr(data, 10)
data['ATR'] = atr

# Calculate the ATR Trailing Stop indicator
src = data['Close']
atr_trailing_stop = atr_trailing_stop(src, atr)
data['ATR_Trailing_Stop'] = atr_trailing_stop


ema = ema(src)
data['EMA'] = ema

def buy_sell_signals(src, xATRTrailingStop, ema):
    above_below =''
    res = []
    last_signal = 0;
    for i in range(len(src)):
        # print(i)
        if i !=0:
            
            if (src[i] > xATRTrailingStop[i] and ema[i] > xATRTrailingStop[i] and ema[i-1]<xATRTrailingStop[i-1]):
                above_below = 'above'
                if last_signal!=1:
                    res.append(1)
                    last_signal = 1
                else:
                    res.append(0)
            elif (src[i] < xATRTrailingStop[i] and ema[i] < xATRTrailingStop[i] and ema[i-1]<xATRTrailingStop[i-1]):
                above_below = 'below'
                if last_signal!=-1:
                    res.append(-1)
                    last_signal = -1
                else:
                    res.append(0)
            else:
                res.append(0)
                last_signal = 0
        if i==0:
            res.append(0)
        
    return res

signals = buy_sell_signals(src, atr_trailing_stop, ema)
data['signal'] = signals

In [54]:
data.set_index('Datetime')

,Open,High,Low,Close,Adj Close,Volume,ATR,ATR_Trailing_Stop,EMA,signal
Datetime,,,,,,,,,,
2023-01-16 00:00:00-05:00,1925.199951,1925.300049,1924.900024,1925.000000,1925.000000,0,NaN,0.000000,1925.000000,0
2023-01-16 00:01:00-05:00,1924.900024,1924.900024,1924.500000,1924.599976,1924.599976,95,NaN,NaN,1924.599976,0
2023-01-16 00:02:00-05:00,1924.599976,1924.599976,1923.900024,1923.900024,1923.900024,132,NaN,NaN,1923.900024,0
2023-01-16 00:03:00-05:00,1923.900024,1924.099976,1923.800049,1923.800049,1923.800049,105,NaN,NaN,1923.800049,0
2023-01-16 00:04:00-05:00,1923.800049,1923.800049,1923.300049,1923.300049,1923.300049,71,NaN,NaN,1923.300049,0
...,...,...,...,...,...,...,...,...,...,...
2023-02-10 16:55:00-05:00,1876.099976,1876.400024,1876.000000,1876.300049,1876.300049,34,0.300012,1876.600061,1876.300049,-1
2023-02-10 16:56:00-05:00,1876.300049,1876.500000,1876.199951,1876.300049,1876.300049,15,0.320020,1876.620068,1876.300049,0
2023-02-10 16:57:00-05:00,1876.300049,1876.599976,1876.099976,1876.199951,1876.199951,58,0.370020,1876.569971,1876.199951,0


In [66]:
RiskRewardRatio = 8
percentage = 0.00001

for i in reversed(range(1, 100)):
    i = i * 0.00001
    def SIGNAL():
        return data.signal
    class MyCandlesStrat(Strategy):  
        def init(self):
            super().init()
            self.signal1 = self.I(SIGNAL)

        def next(self):
            super().next() 
            if self.signal1==1:
                sl1 =self.data.Close[-1]*(1-i)
                tp1 = self.data.Close[-1]*(1+i*RiskRewardRatio)
                # print(self.data.Close[-1],sl1,tp1)
                self.buy(sl=sl1, tp=tp1)
                
            elif self.signal1==-1:
                sl1 =self.data.Close[-1]*(1+i)
                tp1 = self.data.Close[-1]*(1-i*RiskRewardRatio)
                self.sell(sl=sl1, tp=tp1)

    data_new = data.copy()                             # Create copy of pandas DataFrame
    data_new['Datetime'] = pd.to_datetime(data_new['Datetime'])

    data_new = data_new.set_index(pd.DatetimeIndex(data_new['Datetime']))

    bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)
    stat = bt.run()
    print(i, ':' ,stat[8])



C:\Users\tyler\AppData\Local\Temp\ipykernel_34040\1503044343.py:31: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)


0.00099 : 65.17397915686622


C:\Users\tyler\AppData\Local\Temp\ipykernel_34040\1503044343.py:31: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)


0.0009800000000000002 : 68.47595947133897


C:\Users\tyler\AppData\Local\Temp\ipykernel_34040\1503044343.py:31: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)


...

0.00017 : -3.703332034339721


C:\Users\tyler\AppData\Local\Temp\ipykernel_34040\1503044343.py:31: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)


0.00016 : 1.07502694262251


C:\Users\tyler\AppData\Local\Temp\ipykernel_34040\1503044343.py:31: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)


KeyboardInterrupt: 

In [72]:
RiskRewardRatio = 8
percentage = 0.00031

In [73]:
from backtesting import Strategy
def SIGNAL():
    return data.signal
class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==1:
            sl1 =self.data.Close[-1]*(1-percentage)
            tp1 = self.data.Close[-1]*(1+percentage*RiskRewardRatio)
            # print(self.data.Close[-1],sl1,tp1)
            self.buy(sl=sl1, tp=tp1)
            
        elif self.signal1==-1:
            sl1 =self.data.Close[-1]*(1+percentage)
            tp1 = self.data.Close[-1]*(1-percentage*RiskRewardRatio)
            self.sell(sl=sl1, tp=tp1)

In [74]:

data_new = data.copy()                             # Create copy of pandas DataFrame
data_new['Datetime'] = pd.to_datetime(data_new['Datetime'])

In [75]:
data_new = data_new.set_index(pd.DatetimeIndex(data_new['Datetime']))
data_new

,Datetime,Open,High,Low,Close,Adj Close,Volume,ATR,ATR_Trailing_Stop,EMA,signal
Datetime,,,,,,,,,,,
2023-01-16 00:00:00-05:00,2023-01-16 00:00:00-05:00,1925.199951,1925.300049,1924.900024,1925.000000,1925.000000,0,NaN,0.000000,1925.000000,0
2023-01-16 00:01:00-05:00,2023-01-16 00:01:00-05:00,1924.900024,1924.900024,1924.500000,1924.599976,1924.599976,95,NaN,NaN,1924.599976,0
2023-01-16 00:02:00-05:00,2023-01-16 00:02:00-05:00,1924.599976,1924.599976,1923.900024,1923.900024,1923.900024,132,NaN,NaN,1923.900024,0
2023-01-16 00:03:00-05:00,2023-01-16 00:03:00-05:00,1923.900024,1924.099976,1923.800049,1923.800049,1923.800049,105,NaN,NaN,1923.800049,0
2023-01-16 00:04:00-05:00,2023-01-16 00:04:00-05:00,1923.800049,1923.800049,1923.300049,1923.300049,1923.300049,71,NaN,NaN,1923.300049,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-02-10 16:55:00-05:00,2023-02-10 16:55:00-05:00,1876.099976,1876.400024,1876.000000,1876.300049,1876.300049,34,0.300012,1876.600061,1876.300049,-1
2023-02-10 16:56:00-05:00,2023-02-10 16:56:00-05:00,1876.300049,1876.500000,1876.199951,1876.300049,1876.300049,15,0.320020,1876.620068,1876.300049,0
2023-02-10 16:57:00-05:00,2023-02-10 16:57:00-05:00,1876.300049,1876.599976,1876.099976,1876.199951,1876.199951,58,0.370020,1876.569971,1876.199951,0


In [76]:
from backtesting import Backtest

bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)
stat = bt.run()
print(stat)

C:\Users\tyler\AppData\Local\Temp\ipykernel_34040\1233701645.py:3: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)


Start                     2023-01-16 00:00...
End                       2023-02-10 16:59...
Duration                     25 days 16:59:00
Exposure Time [%]                   93.538225
Equity Final [$]               1157938.974715
Equity Peak [$]                1165306.776386
Return [%]                          15.793897
Buy & Hold Return [%]               -2.524674
Return (Ann.) [%]                  710.707185
Volatility (Ann.) [%]              210.528232
Sharpe Ratio                         3.375828
Sortino Ratio                       72.471242
Calmar Ratio                       182.131929
Max. Drawdown [%]                   -3.902156
Avg. Drawdown [%]                   -0.135909
Max. Drawdown Duration        4 days 22:39:00
Avg. Drawdown Duration        0 days 02:40:00
# Trades                                 3151
Win Rate [%]                         31.16471
Best Trade [%]                        2.68926
Worst Trade [%]                     -0.850759
Avg. Trade [%]                    

In [45]:
print(stat)

Start                     2023-01-16 00:00...
End                       2023-02-10 16:59...
Duration                     25 days 16:59:00
Exposure Time [%]                   99.249057
Equity Final [$]                 954267.75376
Equity Peak [$]                1033993.193356
Return [%]                          -4.573225
Buy & Hold Return [%]               -2.524674
Return (Ann.) [%]                  -53.950257
Volatility (Ann.) [%]                 12.4782
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.374944
Avg. Drawdown [%]                   -0.230929
Max. Drawdown Duration       10 days 09:20:00
Avg. Drawdown Duration        0 days 07:48:00
# Trades                                 1332
Win Rate [%]                          49.6997
Best Trade [%]                       2.845318
Worst Trade [%]                     -0.923419
Avg. Trade [%]                    

In [16]:
import matplotlib.pyplot as plt
bt.plot()

C:\Users\tyler\anaconda3\lib\site-packages\backtesting\_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '5T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


ValueError: Length of values (4) does not match length of index (2)